In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import altair as alt
from math import sqrt
from scipy.stats import pearsonr, rankdata
from pandas._testing import assert_series_equal

import pandamonium

pd.set_option("precision", 10)

In [3]:
pd.__version__

'1.1.0'

In [4]:
data = pd.read_json("https://raw.githubusercontent.com/vega/vega-datasets/master/data/penguins.json")

In [5]:
data.head()

,Species,Island,Beak Length (mm),Beak Depth (mm),Flipper Length (mm),Body Mass (g),Sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,None
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE


In [6]:
data.shape

(344, 7)

In [7]:
data = data.clean.column_names()

In [8]:
data.head()

,species,island,beak_length_mm,beak_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,None
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE


# Pearson's correlation

In [9]:
def pearson_corr(x, y):
    mean_x = x.mean()
    mean_y = y.mean()

    xm = x - mean_x
    ym = y - mean_y

    numerator = (xm * ym).sum()
    denominator = sqrt((xm ** 2).sum()) * sqrt((ym ** 2).sum())
    
    return numerator / denominator

In [25]:
pearson_manual = data.corr(method=pearson_corr)
pearson_manual

,beak_length_mm,beak_depth_mm,flipper_length_mm,body_mass_g
beak_length_mm,1.0000000000,-0.2350528704,0.6561813407,0.5951098244
beak_depth_mm,-0.2350528704,1.0000000000,-0.5838512165,-0.4719156212
flipper_length_mm,0.6561813407,-0.5838512165,1.0000000000,0.8712017673
body_mass_g,0.5951098244,-0.4719156212,0.8712017673,1.0000000000


In [26]:
pearson = data.corr(method="pearson")
pearson

,beak_length_mm,beak_depth_mm,flipper_length_mm,body_mass_g
beak_length_mm,1.0000000000,-0.2350528704,0.6561813407,0.5951098244
beak_depth_mm,-0.2350528704,1.0000000000,-0.5838512165,-0.4719156212
flipper_length_mm,0.6561813407,-0.5838512165,1.0000000000,0.8712017673
body_mass_g,0.5951098244,-0.4719156212,0.8712017673,1.0000000000


In [28]:
pearson.compare(pearson_manual)

beak_length_mm               beak_depth_mm                \
                            self         other          self         other   
beak_length_mm               NaN           NaN -0.2350528704 -0.2350528704   
beak_depth_mm      -0.2350528704 -0.2350528704           NaN           NaN   
flipper_length_mm   0.6561813407  0.6561813407 -0.5838512165 -0.5838512165   
body_mass_g         0.5951098244  0.5951098244 -0.4719156212 -0.4719156212   

                  flipper_length_mm                 body_mass_g                
                               self         other          self         other  
beak_length_mm         0.6561813407  0.6561813407  0.5951098244  0.5951098244  
beak_depth_mm         -0.5838512165 -0.5838512165 -0.4719156212 -0.4719156212  
flipper_length_mm               NaN           NaN           NaN           NaN  
body_mass_g                     NaN           NaN           NaN           NaN

In [13]:
data_nan = data.dropna(subset=["beak_length_mm", "beak_depth_mm"])

round(pearsonr(data_nan["beak_length_mm"], data_nan["beak_depth_mm"])[0], 10)

-0.2350528704

# Spearman's rank correlation

The Pearson correlation coefficient between the rank variables.

In [48]:
def compare_rank_data_methods(data, col, method="first"):
    df_compare = pd.DataFrame()

    data_nan = data.dropna(subset=[col])
    df_compare[col] = data_nan[col]

    df_compare["pandas"] = data_nan[col].rank(
        method="first" if method == "ordinal" else method
    )
    
    df_compare["scipy"] = rankdata(
        data_nan[col], method="ordinal" if method == "first" else method
    )

    assert_series_equal(
        df_compare["pandas"], df_compare["scipy"], check_dtype=False, check_names=False
    )

    return df_compare

In [67]:
compare_rank_data_methods(data, "beak_length_mm").sort_values("pandas").head(10)

,beak_length_mm,pandas,scipy
142,32.1,1.0,1
98,33.1,2.0,2
70,33.5,3.0,3
92,34.0,4.0,4
8,34.1,5.0,5
18,34.4,6.0,6
54,34.5,7.0,7
14,34.6,8.0,8
80,34.6,9.0,9
52,35.0,10.0,10


In [68]:
def ordinal_rank(start=1):
    df = pd.DataFrame()
    d = data.dropna(subset=["beak_length_mm"])["beak_length_mm"].tolist()

    df = pd.DataFrame(
        [(i[0], i[1]) for i in sorted(enumerate(d, start=1), key=lambda x: x[1])],
        columns=["original_index", "beak_length_mm"],
    )

    return df

In [69]:
ordinal_rank()

,original_index,beak_length_mm
0,142,32.1
1,98,33.1
2,70,33.5
3,92,34.0
4,8,34.1
...,...,...
337,335,55.1
338,215,55.8
339,321,55.9
340,169,58.0
